In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
mainData = pd.DataFrame()

In [3]:
mainData['Outlook'] = [
    'sunny', 'sunny', 'overcast', 'rain', 'rain', 'rain',
    'overcast', 'sunny', 'sunny', 'rain', 'sunny',
    'overcast', 'overcast', 'rain'
]

In [4]:
mainData['Temperature'] = [
    'hot', 'hot', 'hot', 'mild', 'cool', 'cool', 'cool',
    'mild', 'cool', 'mild', 'mild', 'mild', 'hot', 'mild',
]

In [5]:
mainData['Humidity'] = [
    'high', 'high', 'high', 'high', 'normal', 'normal', 'normal', 
    'high', 'normal', 'normal', 'normal', 'high', 'normal', 'high'
]

In [6]:
mainData['Windy'] = [
    'Weak', 'Strong', 'Weak', 'Weak', 'Weak', 'Strong', 'Strong', 'Weak', 'Weak', 'Weak',
    'Strong', 'Strong', 'Weak', 'Strong'
]

In [7]:
mainData['Decision'] = [
    'N', 'N', 'P', 'P', 'P', 'N', 'P', 'N', 'P', 'P',
    'P', 'P', 'P', 'N'
]

In [8]:
# Current Our Data
mainData

,Outlook,Temperature,Humidity,Windy,Decision
0,sunny,hot,high,Weak,N
1,sunny,hot,high,Strong,N
2,overcast,hot,high,Weak,P
3,rain,mild,high,Weak,P
4,rain,cool,normal,Weak,P
5,rain,cool,normal,Strong,N
6,overcast,cool,normal,Strong,P
7,sunny,mild,high,Weak,N
8,sunny,cool,normal,Weak,P
9,rain,mild,normal,Weak,P


In [9]:
#classify data by each decision
#we use len() to returns numbers of object
#formulas: count_of_data_by_specified_decision / total_of_data
pData = len(mainData.loc[mainData.Decision == 'P']) / len(mainData)
nData = len(mainData.loc[mainData.Decision == 'N']) / len(mainData)

print("P:",pData," , N",nData)

P: 0.6428571428571429  , N 0.35714285714285715


In [10]:
#do entropy decision by results of each decision
#formulas in pdf
entropyDecision = -nData * np.log2(nData) - pData * np.log2(pData)
print(f'H(S) = {entropyDecision:0.4f}')

H(S) = 0.9403


In [26]:
#lets create into functions
def f_entropy_decision(data):
    pData = len(data.loc[data.Decision == 'P']) / len(data)
    nData = len(data.loc[data.Decision == 'N']) / len(data)
    return -nData * np.log2(nData) - pData * np.log2(pData)

In [27]:
#lets try find gain in outlook data
IGDecisionOutlook = f_entropy_decision(mainData)
overallEqn = 'Gain(Decision, Outlook) = Entropy(Decision)'

for name, Outlook in mainData.groupby('Outlook'):
    pOutlookData = len(Outlook.loc[Outlook.Decision == 'P'])
    notPOutlookData = len(Outlook.loc[Outlook.Decision != 'P'])
    totalOutlookData = len(Outlook)
    print(f'p(Decision=P|Outlook={name}) = {pOutlookData}/{totalOutlookData}')
    print(f'p(Decision=N|Outlook={name}) = {notPOutlookData}/{totalOutlookData}')    
    print(f'p(Outlook={name}) = {totalOutlookData}/{len(mainData)}')
    print(f'Entropy(Decision|Outlook={name}) = 'f'-{pOutlookData}/{totalOutlookData}.log2({pOutlookData}/{totalOutlookData}) - 'f'{notPOutlookData}/{totalOutlookData}.log2({notPOutlookData}/{totalOutlookData})')
    entropyDecisionOutlook = 0
    if pOutlookData != 0 :
        entropyDecisionOutlook = (pOutlookData / totalOutlookData) * np.log2(pOutlookData / totalOutlookData)
    
    if notPOutlookData != 0 :
        entropyDecisionOutlook = (notPOutlookData / totalOutlookData) * np.log2(notPOutlookData / totalOutlookData)
    
    IGDecisionOutlook -= (totalOutlookData / len(mainData)) * entropyDecisionOutlook
    overallEqn += f' - p(Outlook={name}).'
    overallEqn += f'Entropy(Decision|Outlook={name})'

print(overallEqn)
print(f'Gain(Decision, Outlook) = {IGDecisionOutlook:0.4f}')

p(Decision=P|Outlook=overcast) = 4/4
p(Decision=N|Outlook=overcast) = 0/4
p(Outlook=overcast) = 4/14
Entropy(Decision|Outlook=overcast) = -4/4.log2(4/4) - 0/4.log2(0/4)
p(Decision=P|Outlook=rain) = 3/5
p(Decision=N|Outlook=rain) = 2/5
p(Outlook=rain) = 5/14
Entropy(Decision|Outlook=rain) = -3/5.log2(3/5) - 2/5.log2(2/5)
p(Decision=P|Outlook=sunny) = 2/5
p(Decision=N|Outlook=sunny) = 3/5
p(Outlook=sunny) = 5/14
Entropy(Decision|Outlook=sunny) = -2/5.log2(2/5) - 3/5.log2(3/5)
Gain(Decision, Outlook) = Entropy(Decision) - p(Outlook=overcast).Entropy(Decision|Outlook=overcast) - p(Outlook=rain).Entropy(Decision|Outlook=rain) - p(Outlook=sunny).Entropy(Decision|Outlook=sunny)
Gain(Decision, Outlook) = 1.2871


In [28]:
# lets create into functions called IG
def IG(data, column, entDecision=f_entropy_decision(mainData)):
    IGDecision = entDecision
    for name, temp in data.groupby(column):
        pData = len(temp.loc[temp.Decision == 'P']) / len(temp)
        notPData = len(temp.loc[temp.Decision != 'P']) / len(temp)

        entropyDecision = 0

        if pData != 0:
            entropyDecision -= (pData) * np.log2(pData)

        if notPData != 0:
            entropyDecision -= (notPData) * np.log2(notPData)

        IGDecision -= (len(temp) / len(mainData)) * entropyDecision
    return IGDecision

In [29]:
#finally lets use it
for col in mainData.columns[:-1]:
    print(f'Gain(Decision, {col}) = {IG(mainData, col):0.4f}')

Gain(Decision, Outlook) = 0.2467
Gain(Decision, Temperature) = 0.0292
Gain(Decision, Humidity) = 0.1518
Gain(Decision, Windy) = 0.0481


In [31]:
for name, temp in mainData.groupby('Outlook'):
    print('-' * 15)
    print(name)
    print('-' * 15)
    print(temp)
    print('-' * 15)

---------------
overcast
---------------
     Outlook Temperature Humidity   Windy Decision
2   overcast         hot     high    Weak        P
6   overcast        cool   normal  Strong        P
11  overcast        mild     high  Strong        P
12  overcast         hot   normal    Weak        P
---------------
---------------
rain
---------------
   Outlook Temperature Humidity   Windy Decision
3     rain        mild     high    Weak        P
4     rain        cool   normal    Weak        P
5     rain        cool   normal  Strong        N
9     rain        mild   normal    Weak        P
13    rain        mild     high  Strong        N
---------------
---------------
sunny
---------------
   Outlook Temperature Humidity   Windy Decision
0    sunny         hot     high    Weak        N
1    sunny         hot     high  Strong        N
7    sunny        mild     high    Weak        N
8    sunny        cool   normal    Weak        P
10   sunny        mild   normal  Strong        P
---------

In [32]:
#so we have done with overcast's decision
#now lets do for another outlooks
#first, lets throw away overcast
nextData = mainData.loc[mainData.Outlook != 'overcast']
nextData

,Outlook,Temperature,Humidity,Windy,Decision
0,sunny,hot,high,Weak,N
1,sunny,hot,high,Strong,N
3,rain,mild,high,Weak,P
4,rain,cool,normal,Weak,P
5,rain,cool,normal,Strong,N
7,sunny,mild,high,Weak,N
8,sunny,cool,normal,Weak,P
9,rain,mild,normal,Weak,P
10,sunny,mild,normal,Strong,P
13,rain,mild,high,Strong,N


In [33]:
#lets begin with sunny outlooks
sunnyData = nextData.loc[nextData.Outlook == 'sunny']

In [34]:
#lets just use ours functions! :D
entropyDecision = f_entropy_decision(sunnyData)
entropyDecision

0.9709505944546686

In [35]:
for col in sunnyData.columns[1:-1]:
    print(f'Gain(Decision, {col}) = {IG(sunnyData, col, entropyDecision):0.4f}')

Gain(Decision, Temperature) = 0.8281
Gain(Decision, Humidity) = 0.9710
Gain(Decision, Windy) = 0.6313


In [36]:
for name, temp in sunnyData.groupby('Humidity'):
    print('-' * 15)
    print(name)
    print('-' * 15)
    print(temp)
    print('-' * 15)

---------------
high
---------------
  Outlook Temperature Humidity   Windy Decision
0   sunny         hot     high    Weak        N
1   sunny         hot     high  Strong        N
7   sunny        mild     high    Weak        N
---------------
---------------
normal
---------------
   Outlook Temperature Humidity   Windy Decision
8    sunny        cool   normal    Weak        P
10   sunny        mild   normal  Strong        P
---------------


In [37]:
#ok, now for rain's outlooks
rainData = nextData.loc[nextData.Outlook == 'rain']
entropyDecision = f_entropy_decision(rainData)
entropyDecision

0.9709505944546686

In [38]:
for col in rainData.columns[1:-1]:
    print(f'Gain(Decision, {col}) = {IG(rainData, col, entropyDecision):0.4f}')

Gain(Decision, Temperature) = 0.6313
Gain(Decision, Humidity) = 0.6313
Gain(Decision, Windy) = 0.9710


In [40]:
for name, temp in rainData.groupby('Windy'):
    print('-' * 15)
    print(name)
    print('-' * 15)
    print(temp)
    print('-' * 15)

---------------
Strong
---------------
   Outlook Temperature Humidity   Windy Decision
5     rain        cool   normal  Strong        N
13    rain        mild     high  Strong        N
---------------
---------------
Weak
---------------
  Outlook Temperature Humidity Windy Decision
3    rain        mild     high  Weak        P
4    rain        cool   normal  Weak        P
9    rain        mild   normal  Weak        P
---------------


In [ ]:
#voila! thats all